In [60]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import jupyter_util as util
from importlib import reload
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/aries-acapy-plugin/controller_acapy/jupyter_util.py'>

## Enter Invite Url:

In [61]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiOTljZjBlZTEtYjY3ZC00OWY4LTlmZmEtNDM5ZjlkMmQ2NmU4IiwgImxhYmVsIjogIkFnZW50IEJvbmQgdGhlIEZpcnN0IChhZG1pbikiLCAicmVjaXBpZW50S2V5cyI6IFsiN21mOUJaOUw0NW04UGc1NUYzN3QzMm9XQ3g0dFVlVDJrbmVoWExWS21BUDQiXSwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQxLmxvY2FsaG9zdCJ9


In [62]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNTBkYWJhYzEtYjI0MS00ZTBkLTk4OWMtMzFkZjZhODE0MDAzIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJsYWJlbCI6ICJBbGljZSBKb25lcyAoYWRtaW4pIiwgInJlY2lwaWVudEtleXMiOiBbIkg2c1huekUzeFVSRHlCUlRMRzg3a3U0WWZRRkFTekxVUjlLbVhyQmQyZllGIl19


In [63]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': '50dabac1-b241-4e0d-989c-31df6a814003',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Alice Jones (admin)',
 'recipientKeys': ['H6sXnzE3xURDyBRTLG87ku4YfQFASzLUR9KmXrBd2fYF'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [64]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 2eyUvLm4qb1YND3RxA1E6jRhzPiwxmT1YLgGb9Y7se4TD2j61BmbjVpKr6jZbVcjyzrUiGEL71aW75NSxED74bVq

Public key: 
 DjFN6jg5SAAMSKJvb6TvVkt75anM7cSATseBZWK3W9r7


**Sending a connection request to acapy**

In [65]:
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [66]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [67]:
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

In [68]:
connection = util.connectAgents(agents[0], agents[1])
connection[1] = connection[1]['raw_repr']['connection_id']
connection[0] = connection[0]['connection_id']
print(connection[0], "\n", connection[1])

fac3731e-8cc6-4226-8046-aa25b2304c4b 
 abf45fbf-0998-40f9-9bbb-2e128a12dd59


### FEATURE DISCOVERY

In [69]:
# consentSchema: function() {
#       return {
#         namespace: "consent",
#         DRI: "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s"
#       }
#     },

# consentAnswers: function() {
#       return [
#         "zQmNeoZGfo1chKACu9RLrVoG4afjLrGnAz1rhsfvGZwg9Z3",
#         "zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"
#       ]

In [70]:
add = requests.post('http://localhost:8151/verifiable-services/add', json={
  "label": "consent_test",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string"
  },
  "consent_schema": {
    "oca_schema_namespace": "consent",
    "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s",
    "data_dri": "zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"
  }
})
add = json.loads(add.text)

newDid1 = util.createDid(agents[1])
newDid0 = util.createDid(agents[0])

try:
    util.registerDidOnLedger(agents[1], newDid1['result']['did'], newDid1['result']['verkey'])
except:
    pass
try:
    util.registerDidOnLedger(agents[0], newDid0['result']['did'], newDid0['result']['verkey'])
except:
    pass

util.setPublicDid(agents[1], newDid1['result']['did'])
util.setPublicDid(agents[0], newDid0['result']['did'])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '@id': '6d295edb-7cb8-4fcc-b206-85e4c4b129fe',
 '~thread': {'thid': '7f5e7865-38b5-4ae0-ba08-003b2a0bd0ca'},
 'result': {'metadata': {'public': True},
  'verkey': '4h6fuP5FDHsVq4Sdh3oxfN2cvAXyVG7GkVDB2x3GmDvw',
  'did': '7mpJAwgyGir27gvGXnBhpo'}}

In [71]:
service_list = requests.get('http://localhost:8150/verifiable-services/DEBUGrequest/' + connection[0])
print(service_list)
service_list = json.loads(service_list.text)
pp.pprint(service_list)

<Response [200]>
{'connection_id': 'fac3731e-8cc6-4226-8046-aa25b2304c4b',
 'created_at': '2020-10-01 10:54:15.870430Z',
 'services': [{'consent_schema': {'data_dri': 'zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G',
                                  'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s',
                                  'oca_schema_namespace': 'consent'},
               'label': 'consent_test',
               'service_id': 'be7a59d1-1a6f-4784-bfd5-33734725d8cd',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}}],
 'updated_at': '2020-10-01 10:54:15.870430Z'}


In [72]:
apply = requests.post('http://localhost:8150/verifiable-services/apply', json={
  "connection_id": connection[0],
  "service_id": service_list['services'][0]['service_id'],
  "payload": "abc",
  "service": service_list['services'][0],
})
apply = json.loads(apply.text)
print(apply)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/verifiable-services/1.0/application', '@id': '7bb9b480-b045-4179-8f89-36a27f00ead0', 'exchange_id': 'e98ff6dc-4105-4131-af13-f68fa67d68df', 'data_dri': 'ddbcc71964f4fd7744bf2d23c031516fb9bd6f766909090044cdc3e4201cd4d8', 'service_id': 'be7a59d1-1a6f-4784-bfd5-33734725d8cd', 'credential_definition_id': '7mpJAwgyGir27gvGXnBhpo:3:CL:1293:consent_schema'}


In [81]:
issue_self = requests.post('http://localhost:8151/verifiable-services/get-issue-self', json={
    "exchange_id": apply['exchange_id'],
})
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

issue_self = requests.post('http://localhost:8151/verifiable-services/process-application', json={
    "issue_id": issue_self[0]['issue_id'],
    "decision": "accept",
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

[{'author': 'other',
  'connection_id': 'abf45fbf-0998-40f9-9bbb-2e128a12dd59',
  'consent_schema': '{"oca_schema_namespace": "consent", "oca_schema_dri": '
                    '"fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s", '
                    '"data_dri": '
                    '"zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"}',
  'created_at': '2020-10-01 10:54:32.739200Z',
  'exchange_id': 'e98ff6dc-4105-4131-af13-f68fa67d68df',
  'issue_id': '852db2726e355088c5d1e04959c2bcb57009a5621a64723742ce4a851caa4ba7',
  'label': 'consent_test',
  'payload': None,
  'service_id': 'be7a59d1-1a6f-4784-bfd5-33734725d8cd',
  'service_schema': '{"oca_schema_namespace": "string", "oca_schema_dri": '
                    '"string"}',
  'state': 'accepted',
  'updated_at': '2020-10-01 10:54:51.814837Z'}]
<Response [200]>
{'credential_exchange_record': {'auto_issue': True,
                                'auto_offer': False,
                                'auto_remove': False,
                   

In [82]:
credentials = requests.get('http://localhost:8150/credentials')
credentials = json.loads(credentials.text) 
pp.pprint(credentials)

{'results': [{'attrs': {'data_dri': 'ddbcc71964f4fd7744bf2d23c031516fb9bd6f766909090044cdc3e4201cd4d8',
                        'oca_schema_dri': 'string',
                        'oca_schema_namespace': 'string'},
              'cred_def_id': '8pX1utXEav1kgqb4U4275X:3:CL:1295:Services',
              'cred_rev_id': None,
              'referent': '0c0e3ab5-b44f-4d82-9fd9-8be78bba94d7',
              'rev_reg_id': None,
              'schema_id': '8pX1utXEav1kgqb4U4275X:2:consent_test:1.0'},
             {'attrs': {'data_dri': 'ddbcc71964f4fd7744bf2d23c031516fb9bd6f766909090044cdc3e4201cd4d8',
                        'oca_schema_dri': 'string',
                        'oca_schema_namespace': 'string'},
              'cred_def_id': '8pX1utXEav1kgqb4U4275X:3:CL:1295:Services',
              'cred_rev_id': None,
              'referent': '8e52d3f0-659c-4677-ad75-371236506873',
              'rev_reg_id': None,
              'schema_id': '8pX1utXEav1kgqb4U4275X:2:consent_test:1.0'}]}


In [83]:
proof_schema = {
  "proof_request": {
    "version": "1.0",
    "name": "Proof request",
    "requested_attributes": {
      "data_dri": {
        "restrictions": [
          {
            "cred_def_id": credentials['results'][0]['cred_def_id'],
          }
        ],
        "name": "data_dri"
      },
    },
    "requested_predicates": {
    },
    "nonce": "12345"
  },
  "connection_id": connection[1],
}
proof_json = json.dumps(proof_schema)

proof = requests.post('http://localhost:8151/present-proof/send-request', json=proof_schema)
proof = json.loads(proof.text)
print(proof)

{'auto_present': False, 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': '8d49fca8-b018-49a2-9aae-67896371de6c', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJ2ZXJzaW9uIjogIjEuMCIsICJuYW1lIjogIlByb29mIHJlcXVlc3QiLCAicmVxdWVzdGVkX2F0dHJpYnV0ZXMiOiB7ImRhdGFfZHJpIjogeyJyZXN0cmljdGlvbnMiOiBbeyJjcmVkX2RlZl9pZCI6ICI4cFgxdXRYRWF2MWtncWI0VTQyNzVYOjM6Q0w6MTI5NTpTZXJ2aWNlcyJ9XSwgIm5hbWUiOiAiZGF0YV9kcmkifX0sICJyZXF1ZXN0ZWRfcHJlZGljYXRlcyI6IHt9LCAibm9uY2UiOiAiMTIzNDUifQ=='}}]}, 'presentation_exchange_id': 'dd79640e-c9b7-466b-b85e-b91cba502d55', 'presentation_request': {'version': '1.0', 'name': 'Proof request', 'requested_attributes': {'data_dri': {'restrictions': [{'cred_def_id': '8pX1utXEav1kgqb4U4275X:3:CL:1295:Services'}], 'name': 'data_dri'}}, 'requested_predicates': {}, 'nonce': '12345'}, 'connection_id': 'abf45fbf-0998-40f9-9bbb-2e12

In [84]:
proof = requests.get('http://localhost:8150/present-proof/records?state=request_received')
proof_requests = json.loads(proof.text)['results']
proof_request = proof_requests[0]
pp.pprint(proof_request)

{'connection_id': 'fac3731e-8cc6-4226-8046-aa25b2304c4b',
 'created_at': '2020-10-01 10:55:21.777931Z',
 'initiator': 'external',
 'presentation_exchange_id': 'f888a71d-57db-40df-8ec7-150fbae6758a',
 'presentation_request': {'name': 'Proof request',
                          'nonce': '12345',
                          'requested_attributes': {'data_dri': {'name': 'data_dri',
                                                                'restrictions': [{'cred_def_id': '8pX1utXEav1kgqb4U4275X:3:CL:1295:Services'}]}},
                          'requested_predicates': {},
                          'version': '1.0'},
 'role': 'prover',
 'state': 'request_received',
 'thread_id': '8d49fca8-b018-49a2-9aae-67896371de6c',
 'trace': False,
 'updated_at': '2020-10-01 10:55:21.777931Z'}


In [85]:
url = 'http://localhost:8150/present-proof/records/' + \
       proof_request['presentation_exchange_id'] + \
       '/send-presentation'

presentation_schema = {
  "requested_attributes": {
    "data_dri": {
      "cred_id": credentials['results'][0]['referent'],
        "revealed": True
    },
  },
   "requested_predicates": {
  },
  "self_attested_attributes": {
      
  }
}
print(presentation_schema)
present = requests.post(url, json=presentation_schema)

print(present.text)

{'requested_attributes': {'data_dri': {'cred_id': '0c0e3ab5-b44f-4d82-9fd9-8be78bba94d7', 'revealed': True}}, 'requested_predicates': {}, 'self_attested_attributes': {}}
{"trace": false, "connection_id": "fac3731e-8cc6-4226-8046-aa25b2304c4b", "thread_id": "8d49fca8-b018-49a2-9aae-67896371de6c", "role": "prover", "updated_at": "2020-10-01 10:55:23.750371Z", "created_at": "2020-10-01 10:55:21.777931Z", "state": "presentation_sent", "presentation": {"proof": {"proofs": [{"primary_proof": {"eq_proof": {"revealed_attrs": {"data_dri": "30586878791154781151037835533422175090933089741194177308066356351013299802157"}, "a_prime": "2065987322032623778068373246795589432889585114674303068431292812356473871674743583322535404475168943834616630030128355006519503444234431499792852085479997110701306420224640734180539409051516197661200159756412130861553757558144859132189911733620042078585552036793772728629994199123539130599914575508708344471675268587282806631528330104328224820470252285262914243673654772

In [47]:
# proof_schema = {
#   "proof_request": {
#     "version": "1.0",
#     "name": "Proof request",
#     "requested_attributes": {
#       "additionalProp1": {
#         "non_revoked": {
#           "from_epoch": 1600672361,
#           "to_epoch": 1600672361
#         },
#         "restrictions": [
#           {
#             "credential_definition_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#             "schema_version": "1.0",
#             "issuer_did": "WgWxqztrNooG92RXvxSTWv",
#             "schema_name": "transcript",
#             "schema_issuer_did": "WgWxqztrNooG92RXvxSTWv",
#             "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#             "schema_id": "WgWxqztrNooG92RXvxSTWv:2:schema_name:1.0"
#           }
#         ],
#         "name": "favouriteDrink"
#       },
#       "additionalProp2": {
#         "non_revoked": {
#           "from_epoch": 1600672361,
#           "to_epoch": 1600672361
#         },
#         "restrictions": [
#           {
#             "credential_definition_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#             "schema_version": "1.0",
#             "issuer_did": "WgWxqztrNooG92RXvxSTWv",
#             "schema_name": "transcript",
#             "schema_issuer_did": "WgWxqztrNooG92RXvxSTWv",
#             "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#             "schema_id": "WgWxqztrNooG92RXvxSTWv:2:schema_name:1.0"
#           }
#         ],
#         "name": "favouriteDrink"
#       },
#       "additionalProp3": {
#         "non_revoked": {
#           "from_epoch": 1600672361,
#           "to_epoch": 1600672361
#         },
#         "restrictions": [
#           {
#             "credential_definition_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#             "schema_version": "1.0",
#             "issuer_did": "WgWxqztrNooG92RXvxSTWv",
#             "schema_name": "transcript",
#             "schema_issuer_did": "WgWxqztrNooG92RXvxSTWv",
#             "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#             "schema_id": "WgWxqztrNooG92RXvxSTWv:2:schema_name:1.0"
#           }
#         ],
#         "name": "favouriteDrink"
#       }
#     },
#     "requested_predicates": {
#       "additionalProp1": {
#         "p_type": ">=",
#         "p_value": 0,
#         "name": "index",
#         "non_revoked": {
#           "from_epoch": 1600672361,
#           "to_epoch": 1600672361
#         },
#         "restrictions": [
#           {
#             "credential_definition_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#             "schema_version": "1.0",
#             "issuer_did": "WgWxqztrNooG92RXvxSTWv",
#             "schema_name": "transcript",
#             "schema_issuer_did": "WgWxqztrNooG92RXvxSTWv",
#             "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#             "schema_id": "WgWxqztrNooG92RXvxSTWv:2:schema_name:1.0"
#           }
#         ]
#       },
#       "additionalProp2": {
#         "p_type": ">=",
#         "p_value": 0,
#         "name": "index",
#         "non_revoked": {
#           "from_epoch": 1600672361,
#           "to_epoch": 1600672361
#         },
#         "restrictions": [
#           {
#             "credential_definition_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#             "schema_version": "1.0",
#             "issuer_did": "WgWxqztrNooG92RXvxSTWv",
#             "schema_name": "transcript",
#             "schema_issuer_did": "WgWxqztrNooG92RXvxSTWv",
#             "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#             "schema_id": "WgWxqztrNooG92RXvxSTWv:2:schema_name:1.0"
#           }
#         ]
#       },
#       "additionalProp3": {
#         "p_type": ">=",
#         "p_value": 0,
#         "name": "index",
#         "non_revoked": {
#           "from_epoch": 1600672361,
#           "to_epoch": 1600672361
#         },
#         "restrictions": [
#           {
#             "credential_definition_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#             "schema_version": "1.0",
#             "issuer_did": "WgWxqztrNooG92RXvxSTWv",
#             "schema_name": "transcript",
#             "schema_issuer_did": "WgWxqztrNooG92RXvxSTWv",
#             "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#             "schema_id": "WgWxqztrNooG92RXvxSTWv:2:schema_name:1.0"
#           }
#         ]
#       }
#     },
#     "nonce": "1234567890"
#   },
#   "connection_id": connection[0],
#   "trace": True,
#   "comment": "string"
# }
# proof_json = json.dumps(proof_schema)

# proof = requests.get('http://localhost:8150/present-proof/send-request', json=proof_json)


{"proof_request": {"version": "1.0", "name": "Proof request", "requested_attributes": {"additionalProp1": {"non_revoked": {"from_epoch": 1600672361, "to_epoch": 1600672361}, "restrictions": [{"credential_definition_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag", "schema_version": "1.0", "issuer_did": "WgWxqztrNooG92RXvxSTWv", "schema_name": "transcript", "schema_issuer_did": "WgWxqztrNooG92RXvxSTWv", "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag", "schema_id": "WgWxqztrNooG92RXvxSTWv:2:schema_name:1.0"}], "name": "favouriteDrink"}, "additionalProp2": {"non_revoked": {"from_epoch": 1600672361, "to_epoch": 1600672361}, "restrictions": [{"credential_definition_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag", "schema_version": "1.0", "issuer_did": "WgWxqztrNooG92RXvxSTWv", "schema_name": "transcript", "schema_issuer_did": "WgWxqztrNooG92RXvxSTWv", "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag", "schema_id": "WgWxqztrNooG92RXvxSTWv:2:schema_name:1.0"}], "name": "favouriteDrink"}, "additional

In [53]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": add['hash_id'],
  "connection_id": connection[1]
})

In [54]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": "ddd",
  "connection_id": connection[1]
})

In [55]:
print(send.text)

{"hash_id": "ddd", "connection_id": "a361d994-aaea-4f5c-9bf4-9644a774de2b", "record_id": "1e61b917-946b-4a56-8810-5b48081f79e6", "exchange_id": "c5f71769-b806-49fe-8861-15f49ffad9e3", "state": "pending"}


### GET Request By RECORD_ID

### GET Request List and filter by state and author

In [56]:
request_list = requests.post('http://localhost:8150/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("Agent1:", )
pp.pprint(request_list)
request_list = requests.post('http://localhost:8151/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("\nAgent2:")
pp.pprint(request_list)

Agent1:
[{'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:17.991960Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:17.991960Z'},
 {'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:16.748526Z',
  'exchange_id': 'f51e4a06-2214-4b9e-a28a-d4c336258f66',
  'payload': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'state': 'approved',
  'updated_at': '2020-07-16 09:09:16.748526Z'}]

Agent2:
[{'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:17.950785Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:18.029400Z'},
 {'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-

### GET RECORD LIST

In [57]:
getRequests = requests.post('http://localhost:8150/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent1: ")
pp.pprint(record_list)

getRequests = requests.post('http://localhost:8151/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent2: ")
pp.pprint(record_list)

Agent1: 
[{'author': 'self',
  'created_at': '2020-07-16 09:09:15.407083Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:15.407083Z'}]
Agent2: 
[{'author': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:16.808608Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:16.808608Z'}]


In [ ]:

# message = util.buildMessage(
#         'https://didcomm.org/issue-credential/1.0/propose-credential'
#     )
# testing = util.sendMessage(message, agents[0])
# pp.pprint(testing)

In [ ]:
# pp.pprint(util.sendSchema(agents[0], "schesssdddvd", "1.0", ["memesssaaaaaaa"]))

In [ ]:
# result = util.schemaGetList(agents[0])
# last_schema_id = result['results'][-1]['schema_id']
# print(result)

In [ ]:
# util.schemaGet(agents[0], last_schema_id)